In [11]:
import pandas as pd
import os
import openpyxl
from openpyxl import load_workbook
from openpyxl.worksheet.datavalidation import DataValidation
from openpyxl.utils import get_column_letter
import os

### Read all file names in the directory

In [12]:
#READ ALL FILE NAMES IN A DIRECTORY
def read_file_names(directory):
    file_names = []
    for file in os.listdir(directory):
        if file.endswith(".xlsx"):
            file_names.append(f"{directory}/{file}")
    return file_names

file_names = read_file_names(r"FINAL-PROGRAMS")

#list all the file names

file_names

['FINAL-PROGRAMS/FINAL- ALS-CLC 2024 as of May 2025.xlsx',
 'FINAL-PROGRAMS/FINAL- ELECTRIFICATION 2017-2024 (as of May)-UPDATED.xlsx',
 'FINAL-PROGRAMS/FINAL- GABALDON 2020-2024 (as of May)-UPDATED.xlsx',
 'FINAL-PROGRAMS/FINAL- HEALTH FACILITIES 2022-2024 (as of May)-UPDATED.xlsx',
 'FINAL-PROGRAMS/FINAL- LMS 2020-2024 (as of May)_UPDATED.xlsx',
 'FINAL-PROGRAMS/FINAL- NEWCON 2014-2024 as of May.xlsx',
 'FINAL-PROGRAMS/FINAL- QRF 2020- 2024 (as of May)-updated.xlsx',
 'FINAL-PROGRAMS/FINAL- REPAIR 2020-2024 as of May- UPDATED.xlsx']

In [13]:
import re

def extract_program_code(file_name):
    # This regex looks for 'FINAL-PROGRAMS/FINAL- ' followed by any non-space characters (including dashes)
    match = re.search(r'FINAL-PROGRAMS/FINAL- ([^\s]+)', file_name)
    if match:
        # Return the matched group, which is the program code
        return match.group(1)
    else:
        # If no match is found, return None or an appropriate message
        return None

# Extract program codes for all file names and store in a list
extracted_prefixes = []
for file_name in file_names:
    code = extract_program_code(file_name)
    extracted_prefixes.append(code)

print(extracted_prefixes)

['ALS-CLC', 'ELECTRIFICATION', 'GABALDON', 'HEALTH', 'LMS', 'NEWCON', 'QRF', 'REPAIR']


### Delete Summary sheets to avoid conflict

In [14]:

delete_summary_sheets = False
if delete_summary_sheets == True:
    for file in file_names:
        wb = load_workbook(file)

        # Define keywords to match against (case-insensitive)
        keywords = ["summary"]

        # Find sheets to delete
        sheets_to_delete = [
            sheet for sheet in wb.sheetnames
            if any(key in sheet.lower() for key in keywords)
        ]

        # Delete the matched sheets
        for sheet in sheets_to_delete:
            std = wb[sheet]
            wb.remove(std)
            print(f"Deleted sheet: {sheet} in file {file}")

        # Save the workbook (overwrites original file)
        wb.save(file)


### Filter programs per region

#### Extract NIR region first for all programs

In [15]:
NIR_divisions = [
    "Bacolod City",
    "Bago City",
    "Bais City",
    "Bayawan City",
    "Cadiz City",
    "Canlaon City",
    "Dumaguete City",
    "Escalante City",
    "Guihulngan City",
    "Himamaylan City",
    "Kabankalan City",
    "La Carlota City",
    "Negros Occidental",
    "Negros Oriental",
    "Sagay City",
    "San Carlos City",
    "Silay City",
    "Sipalay City",
    "Siquijor",
    "Tanjay City",
    "Victorias City"
]



In [16]:
# Load your Excel file
for program, input_file in zip(extracted_prefixes, file_names):
    print(f"Processing program: {program} with input file: {input_file}")
    # wb = load_workbook(input_file)
    # sheet = wb.active
    # print(f"Processing program: {program} with input file: {input_file} in {sheet_name}")

    # Read the Excel file
    master_df = pd.read_excel(input_file)
    
    df = master_df
        
    #format specific columns to short date
    
    # Use "Region" or "REGION" as the region column
    if "Region" in df.columns:
        region_column = "Region"
    elif "REGION" in df.columns:
        region_column = "REGION"
    else:
        raise ValueError("No region column found in the DataFrame.")
    
    if "Division" in df.columns:
        division_column = "Division"
    elif "DIVISION" in df.columns:
        division_column = "DIVISION"
    else:
        raise ValueError("No region column found in the DataFrame.")
    # print(f"Processing program: {program} with input file: {input_file}")

    df.loc[
        (df[division_column].isin(NIR_divisions)) & 
        (df[region_column].isin(['Region VI', 'Region VII'])),
        region_column
    ] = 'NIR'
    



    # Get unique regions
    regions = df[region_column].dropna().unique()
    print(f"{program}: {regions}")

    # Filter and save each region's data
    for region in regions:
        # Filter the DataFrame for the current region
        filtered_df = df[df[region_column] == region]

        # Define the output file name
        region_dir = os.path.join("Filtered_By_Region", region)
        if not os.path.exists(region_dir):
            os.makedirs(region_dir, exist_ok=True)
        output_file = os.path.join(region_dir, f"{region}_{program}.xlsx")

        # Save the filtered DataFrame to an Excel file
        filtered_df.to_excel(output_file, index=False,)
        print(f"Saved {region} data to {output_file}")




Processing program: ALS-CLC with input file: FINAL-PROGRAMS/FINAL- ALS-CLC 2024 as of May 2025.xlsx
ALS-CLC: ['CARAGA' 'Region I' 'Region III' 'Region IV-B']
Saved CARAGA data to Filtered_By_Region\CARAGA\CARAGA_ALS-CLC.xlsx
Saved Region I data to Filtered_By_Region\Region I\Region I_ALS-CLC.xlsx
Saved Region III data to Filtered_By_Region\Region III\Region III_ALS-CLC.xlsx
Saved Region IV-B data to Filtered_By_Region\Region IV-B\Region IV-B_ALS-CLC.xlsx
Processing program: ELECTRIFICATION with input file: FINAL-PROGRAMS/FINAL- ELECTRIFICATION 2017-2024 (as of May)-UPDATED.xlsx


C:\Users\Work - Lawrence\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


ELECTRIFICATION: ['CAR' 'CARAGA' 'Region I' 'Region II' 'Region III' 'Region IV-A'
 'Region IV-B' 'Region IX' 'Region V' 'Region VI' 'NIR' 'Region VII'
 'Region VIII' 'Region X' 'Region XI' 'Region XII' 'NCR']
Saved CAR data to Filtered_By_Region\CAR\CAR_ELECTRIFICATION.xlsx
Saved CARAGA data to Filtered_By_Region\CARAGA\CARAGA_ELECTRIFICATION.xlsx
Saved Region I data to Filtered_By_Region\Region I\Region I_ELECTRIFICATION.xlsx
Saved Region II data to Filtered_By_Region\Region II\Region II_ELECTRIFICATION.xlsx
Saved Region III data to Filtered_By_Region\Region III\Region III_ELECTRIFICATION.xlsx
Saved Region IV-A data to Filtered_By_Region\Region IV-A\Region IV-A_ELECTRIFICATION.xlsx
Saved Region IV-B data to Filtered_By_Region\Region IV-B\Region IV-B_ELECTRIFICATION.xlsx
Saved Region IX data to Filtered_By_Region\Region IX\Region IX_ELECTRIFICATION.xlsx
Saved Region V data to Filtered_By_Region\Region V\Region V_ELECTRIFICATION.xlsx
Saved Region VI data to Filtered_By_Region\Region VI

### Apply Dropdown Options at the final column

In [17]:
from openpyxl import load_workbook
from openpyxl.worksheet.datavalidation import DataValidation
from openpyxl.utils import get_column_letter
import os

In [18]:
# Read all .xlsx files in all subfolders of 'Filtered_By_Region'
all_files = []
for root, dirs, files in os.walk("Filtered_By_Region"):
    for file in files:
        if file.endswith(".xlsx"):
            all_files.append(os.path.join(root, file))

all_files

['Filtered_By_Region\\BARMM\\BARMM_NEWCON.xlsx',
 'Filtered_By_Region\\BARMM\\BARMM_REPAIR.xlsx',
 'Filtered_By_Region\\CAR\\CAR_ELECTRIFICATION.xlsx',
 'Filtered_By_Region\\CAR\\CAR_GABALDON.xlsx',
 'Filtered_By_Region\\CAR\\CAR_HEALTH.xlsx',
 'Filtered_By_Region\\CAR\\CAR_LMS.xlsx',
 'Filtered_By_Region\\CAR\\CAR_NEWCON.xlsx',
 'Filtered_By_Region\\CAR\\CAR_QRF.xlsx',
 'Filtered_By_Region\\CAR\\CAR_REPAIR.xlsx',
 'Filtered_By_Region\\CARAGA\\CARAGA_ALS-CLC.xlsx',
 'Filtered_By_Region\\CARAGA\\CARAGA_ELECTRIFICATION.xlsx',
 'Filtered_By_Region\\CARAGA\\CARAGA_GABALDON.xlsx',
 'Filtered_By_Region\\CARAGA\\CARAGA_HEALTH.xlsx',
 'Filtered_By_Region\\CARAGA\\CARAGA_LMS.xlsx',
 'Filtered_By_Region\\CARAGA\\CARAGA_NEWCON.xlsx',
 'Filtered_By_Region\\CARAGA\\CARAGA_QRF.xlsx',
 'Filtered_By_Region\\CARAGA\\CARAGA_REPAIR.xlsx',
 'Filtered_By_Region\\NCR\\NCR_ELECTRIFICATION.xlsx',
 'Filtered_By_Region\\NCR\\NCR_GABALDON.xlsx',
 'Filtered_By_Region\\NCR\\NCR_HEALTH.xlsx',
 'Filtered_By_Region\\

In [19]:
# Example long list of options
newcon_dropdown_options = [
     r"0% - 10%: Foundation completed: Groundwork finished; no vertical structure yet.",
    r"11% - 25%: Structure and rough-in started: Structural framing in progress; initial MEP rough-in.",
    r"26% - 50%: Structure erected, partial roofing: Building shape defined; roof and systems advancing.",
    r"51% - 75%: Exterior sealed, interior work underway: Enclosed structure; painting, flooring, and testing begin.",
    r"76% - 90%: Final finishes and inspections: Systems tested; ",
    r"91% - 99%: Final touches and punch list: Minor adjustments; final inspections and approvals.",
    r"100% - Construction complete: Ready for handover and occupancy."
]

other_dropdown_options = [
     r"0% - 10%",
    r"11% - 25%",
    r"26% - 50%",
    r"51% - 75%",
    r"76% - 90%",
    r"91% - 99%",
    r"100%"
]

#### Only run this part of code for adding a column
Change value of header to current weak

In [20]:
for file in all_files:
    if "LMS" in file or "NEWCON" in file:
        dropdown = newcon_dropdown_options
    else:
        dropdown = other_dropdown_options
        
    wb = load_workbook(file)
    ws = wb.active

    # Add a new worksheet (or reuse if exists)
    if "DropdownOptions" not in wb.sheetnames:
        dropdown_ws = wb.create_sheet("DropdownOptions")
    else:
        dropdown_ws = wb["DropdownOptions"]
        
    # Write the options into the helper sheet starting at A1
    for i, option in enumerate(dropdown, start=1):
        dropdown_ws.cell(row=i, column=1, value=option)

    # Hide the helper sheet (optional)
    dropdown_ws.sheet_state = 'hidden'

    #  Define the range where options are stored
    options_range = f"DropdownOptions!$A$1:$A${len(dropdown)}"
        
    new_col_index = ws.max_column + 1
    
    ws.insert_cols(new_col_index)
    ws.cell(row=1, column=new_col_index).value = "Status as of July 11, 2025"

    dv = DataValidation(
        type="list",
        formula1=f"={options_range}", # Reference the options in the helper sheet
        allow_blank=True
    )

    # Apply validation
    ws.add_data_validation(dv)
    col_letter = get_column_letter(ws.max_column)
    dv.add(f"{col_letter}2:{col_letter}{ws.max_row}")
    
    #highlight the new column
    for cell in ws[f"{col_letter}"]:
        cell.fill = openpyxl.styles.PatternFill(start_color="FFFF00", end_color="FFFF00", fill_type="solid")
    #format first row, wrap text, align center, and make the text bold
    for cell in ws[1]:
        cell.font = openpyxl.styles.Font(bold=True)
        cell.alignment = openpyxl.styles.Alignment(wrap_text=True, horizontal='center', vertical='center')
        #freeze row 1
        ws.freeze_panes = ws["A2"]  # Freeze the first row

    for row in ws.iter_rows(min_row=1, max_row=ws.max_row, min_col=1, max_col=ws.max_column):
        for cell in row:
            #add border to all cells
            cell.border = openpyxl.styles.Border(
                left=openpyxl.styles.Side(style='thin'),
                right=openpyxl.styles.Side(style='thin'),
                top=openpyxl.styles.Side(style='thin'),
                bottom=openpyxl.styles.Side(style='thin')
            )
    #autofit column width
    for col in ws.columns:
        max_length = 0
        column = col[0].column_letter  # Get the column letter
        for cell in col:
            try:
                if len(str(cell.value)) > max_length:
                    max_length = len(str(cell.value))
            except:
                pass
        adjusted_width = (max_length + 2)
        ws.column_dimensions[column].width = adjusted_width
    # SAVE WORKSHEET
    wb.save(file)
    print(f"Updated {file} with new dropdown options in column {col_letter}.")
    
print("All files updated with dropdown options successfully.")

Updated Filtered_By_Region\BARMM\BARMM_NEWCON.xlsx with new dropdown options in column AC.
Updated Filtered_By_Region\BARMM\BARMM_REPAIR.xlsx with new dropdown options in column AM.
Updated Filtered_By_Region\CAR\CAR_ELECTRIFICATION.xlsx with new dropdown options in column AB.
Updated Filtered_By_Region\CAR\CAR_GABALDON.xlsx with new dropdown options in column AB.
Updated Filtered_By_Region\CAR\CAR_HEALTH.xlsx with new dropdown options in column AA.
Updated Filtered_By_Region\CAR\CAR_LMS.xlsx with new dropdown options in column Z.
Updated Filtered_By_Region\CAR\CAR_NEWCON.xlsx with new dropdown options in column AC.
Updated Filtered_By_Region\CAR\CAR_QRF.xlsx with new dropdown options in column AA.
Updated Filtered_By_Region\CAR\CAR_REPAIR.xlsx with new dropdown options in column AM.
Updated Filtered_By_Region\CARAGA\CARAGA_ALS-CLC.xlsx with new dropdown options in column X.
Updated Filtered_By_Region\CARAGA\CARAGA_ELECTRIFICATION.xlsx with new dropdown options in column AB.
Updated Fi